In [1]:
import pandas as pd
import numpy as np
import os
import json
import pickle
from collections import defaultdict
from datetime import datetime
import openai
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored
openai.api_key = os.environ.get('OPENAI_API_KEY')

In [2]:
user_test_path = '../beauty_data/processed_data/user_test.json'
user_valid_path = '../beauty_data/processed_data/user_valid.json'

In [3]:
with open(user_test_path, 'rb') as f:
    user_test_dict = json.load(f)
print(len(user_test_dict))

624


In [4]:
with open(user_valid_path, 'rb') as f:
    user_valid_dict = json.load(f)
print(len(user_valid_dict))

624


In [5]:
print(user_valid_dict['0'])
print(user_test_dict['0'])

[1, 2, 3, 4, 5, 5]
[2, 3, 4, 5, 5, 6]


In [6]:
item_information_path = '../beauty_data/processed_data/processed_item_information.json'
with open(item_information_path, 'r') as f:
    item_information_dict = json.load(f)
print(len(item_information_dict), item_information_dict['1'])

1216 {'title': 'Monoi - Monoi Pitate Jasmine 4 fl oz', 'brand': 'Monoi', 'price': '', 'description': '\n\nThe product is a jasmine-scented coconut oil, great for dry skin and hair, making them soft and non-oily. However, the jasmine scent may not be natural and not everyone may like it.'}


In [7]:
ratings_path = '../beauty_data/processed_data/ratings.csv'
ratings_df = pd.read_csv(ratings_path)
ratings_df.head()

,item,user,rating,timestamp
0,0,0,4.0,1155081600
1,1,0,5.0,1268697600
2,2,0,5.0,1404604800
3,3,0,5.0,1404604800
4,4,0,5.0,1404604800


In [9]:
user_profile_summary_path = '../beauty_data/processed_data/user_profile_summary_wo_description.json'
with open(user_profile_summary_path, 'r') as f:
    user_profile_summary_dict = json.load(f)
print(len(user_profile_summary_dict), user_profile_summary_dict['1'])

160 
The user disliked 100% Cotton Premium Salon Towels 16&quot; X 27&quot; - Pack of 12 Pcs (Royal Blue) because they were likely not satisfied with the product's quality or size, as they specifically mentioned the dimensions and material. The user seems to prefer high-quality, possibly larger-sized towels.


### Creating zeroshot data

In [10]:
def get_item_description(item_information):
    item_details = ""
    if 'title' in item_information:
        item_details += item_information['title']
    if 'brand' in item_information:
        item_details += f" Brand: {item_information['brand']}"
    if 'price' in item_information:
        item_details += f" Price: {item_information['price']}"
    # if 'description' in item_information:
    #     item_details += f" Description: {item_information['description']}"
    return item_details

In [13]:
### Test Data
zeroshot_dict = dict()
check_missed_items = []
for user, item_list in user_test_dict.items():
    # print(user, item_list)
    user_rating_dict = ratings_df[ratings_df['user'] == int(user)]
    # print(user_rating_dict)
    if user in user_profile_summary_dict:
        prompt = f"""You are an expert beauty product recommender. You are provided with a user's profile and list of recent products and their descriptions that the user purchases and whether the user liked it or disliked it.
        User Profile - {user_profile_summary_dict[user]}
        User purchased the following items in the given order. List of recent items and their description -
        """
    else:
        prompt = """You are an expert beauty product recommender. You are provided with user's list of recent products and their descriptions that the user purchases and whether the user liked it or disliked it.
        User purchased the following items in the given order. List of recent items and their description -
        """
    for item in item_list[:-1]:
        try:
            user_item_rating = user_rating_dict[user_rating_dict['item'] == item]['rating'].values[0]
        except Exception as e:
            print(f"User: {user} Item: {item} error {e}")
            continue
        if user_item_rating > 3:
            ### Liked
            prompt += f"Liked {get_item_description(item_information_dict[str(item)])}\n"
        else:
            prompt += f"Disliked {get_item_description(item_information_dict[str(item)])}\n"
    target_item = item_list[-1]
    if str(target_item) not in item_information_dict:
        continue
    target_item_details = get_item_description(item_information_dict[str(target_item)])
    prompt += f"""As an expert beauty product recommender, do the following steps -
    1. Analyze the user profile, list of products purchased by the user in order and summarize the user behavior by identifying the characteristics he liked and disliked about the products in at most 100 words.
    2. Explain with reasoning why the user will like or dislike the target product - {target_item_details} in atmost 100 words.
    3. Finally, from the reasoning, predict whether the user will like the target product or not in the format - Prediction: Yes or No"""
    zeroshot_dict[user] = prompt
    # break

User: 64 Item: 229 error index 0 is out of bounds for axis 0 with size 0
User: 215 Item: 399 error index 0 is out of bounds for axis 0 with size 0
User: 401 Item: 880 error index 0 is out of bounds for axis 0 with size 0
User: 427 Item: 399 error index 0 is out of bounds for axis 0 with size 0


In [14]:
### Valid Data
zeroshot_valid_dict = dict()
check_missed_items = []
for user, item_list in user_valid_dict.items():
    # print(user, item_list)
    user_rating_dict = ratings_df[ratings_df['user'] == int(user)]
    # print(user_rating_dict)
    if user in user_profile_summary_dict:
        prompt = f"""You are an expert beauty product recommender. You are provided with a user's profile and list of recent products and their descriptions that the user purchases and whether the user liked it or disliked it.
        User Profile - {user_profile_summary_dict[user]}
        User purchased the following items in the given order. List of recent items and their description -
        """
    else:
        prompt = """You are an expert beauty product recommender. You are provided with user's list of recent products and their descriptions that the user purchases and whether the user liked it or disliked it.
        User purchased the following items in the given order. List of recent items and their description -
        """
    for item in item_list[:-1]:
        try:
            user_item_rating = user_rating_dict[user_rating_dict['item'] == item]['rating'].values[0]
        except Exception as e:
            print(f"User: {user} Item: {item} error {e}")
            continue
        if user_item_rating > 3:
            ### Liked
            prompt += f"Liked {get_item_description(item_information_dict[str(item)])}\n"
        else:
            prompt += f"Disliked {get_item_description(item_information_dict[str(item)])}\n"
    target_item = item_list[-1]
    if str(target_item) not in item_information_dict:
        continue
    target_item_details = get_item_description(item_information_dict[str(target_item)])
    prompt += f"""As an expert beauty product recommender, do the following steps -
    1. Analyze the user profile, list of products purchased by the user in order and summarize the user behavior by identifying the characteristics he liked and disliked about the products in at most 100 words.
    2. Explain with reasoning why the user will like or dislike the target product - {target_item_details} in atmost 100 words.
    3. Finally, from the reasoning, predict whether the user will like the target product or not in the format - Prediction: Yes or No"""
    zeroshot_valid_dict[user] = prompt
    # break

User: 64 Item: 229 error index 0 is out of bounds for axis 0 with size 0
User: 215 Item: 399 error index 0 is out of bounds for axis 0 with size 0
User: 401 Item: 880 error index 0 is out of bounds for axis 0 with size 0
User: 427 Item: 399 error index 0 is out of bounds for axis 0 with size 0


In [15]:
print(len(zeroshot_dict), len(zeroshot_valid_dict))

622 624


In [16]:
print(zeroshot_dict['0'])
print("-"*100)
print(zeroshot_valid_dict['0'])

You are an expert beauty product recommender. You are provided with a user's profile and list of recent products and their descriptions that the user purchases and whether the user liked it or disliked it.
        User Profile - 
The user seems to prefer products from niche brands Fresh and Monoi. They like fragrances that are fruity and floral, as evidenced by their preference for Fresh Eau De Parfum EDP in Fig Apricot and Monoi - Monoi Pitate Jasmine. The user's positive feedback suggests that they appreciate high-quality, luxurious scents that are not overpowering. They may also prefer products that are ethically sourced and cruelty-free, as both Fresh and Monoi are known for their commitment to sustainability.
        User purchased the following items in the given order. List of recent items and their description -
        Liked Yardley By Yardley Of London Unisexs Lay It On Thick Hand &amp; Foot Cream 5.3 Oz Brand: Yardley Price: 
Liked Fruits &amp; Passion Blue Refreshing Shower

In [17]:
total_len = 0
for user, prompt in zeroshot_dict.items():
    total_len += len(prompt.strip().split(' '))
print(total_len/len(zeroshot_dict))

294.363344051447


In [18]:
total_len = 0
for user, prompt in zeroshot_valid_dict.items():
    total_len += len(prompt.strip().split(' '))
print(total_len/len(zeroshot_dict))

282.7347266881029


In [23]:
users_very_long = []
for user, prompt in zeroshot_valid_dict.items():
    if len(prompt.strip().split(' ')) > 500:
        users_very_long.append(user)

In [24]:
print(len(users_very_long), users_very_long)

1 ['59']


In [25]:
print(zeroshot_valid_dict['12'])

You are an expert beauty product recommender. You are provided with user's list of recent products and their descriptions that the user purchases and whether the user liked it or disliked it.
        User purchased the following items in the given order. List of recent items and their description -
        Liked Yardley By Yardley Of London Unisexs Lay It On Thick Hand &amp; Foot Cream 5.3 Oz Brand: Yardley Price: 
Liked Fruits &amp; Passion Blue Refreshing Shower Gel - 6.7 fl. oz. Brand: Fruits & Passion Price: 
Liked Bonne Bell Smackers Bath and Body Starburst Collection Brand: Bonne Bell Price: 
Liked Bath &amp; Body Works Ile De Tahiti Moana Coconut Vanille Moana Body Wash with Tamanoi 8.5 oz Brand: Bath & Body Works Price: 
As an expert beauty product recommender, do the following steps -
    1. Analyze the user profile, list of products purchased by the user in order and summarize the user behavior by identifying the characteristics he liked and disliked about the products in at 

In [26]:
for key in users_very_long:
    zeroshot_valid_dict.pop(key, None)
print(len(zeroshot_valid_dict))

623


In [27]:
zeroshot_valid_path = './ctr_zeroshot_dataset/zeroshot_valid.json'
with open(zeroshot_valid_path, 'w+') as f:
    json.dump(zeroshot_valid_dict, f)

In [34]:
users_very_long = []
for user, prompt in zeroshot_dict.items():
    if len(prompt.strip().split(' ')) > 500:
        users_very_long.append(user)

In [35]:
for key in users_very_long:
    zeroshot_dict.pop(key, None)
print(len(zeroshot_dict))

602


In [36]:
zeroshot_test_path = './ctr_zeroshot_dataset/zeroshot_test.json'
with open(zeroshot_test_path, 'w+') as f:
    json.dump(zeroshot_dict, f)

In [37]:
ratings_df[ratings_df['user'] == int('0')]

,item,user,rating,timestamp
0,0,0,4.0,1155081600
1,1,0,5.0,1268697600
2,2,0,5.0,1404604800
3,3,0,5.0,1404604800
4,4,0,5.0,1404604800
5,5,0,5.0,1404604800
6,5,0,5.0,1404604800
7,6,0,5.0,1461801600


In [38]:
test_prompt = zeroshot_dict['34']
print(test_prompt)
print("-"*100)
response = openai.completions.create(
    # model = "gpt-3.5-turbo-0125",
    model="gpt-3.5-turbo-instruct",
    prompt = test_prompt,
    temperature=0.3,
    max_tokens=256,
    # top_p=0.3,
    # frequency_penalty=0.5,
    # presence_penalty=0.5
)
print(response.choices[0].text)

You are an expert beauty product recommender. You are provided with user's list of recent products and their descriptions that the user purchases and whether the user liked it or disliked it.
        User purchased the following items in the given order. List of recent items and their description -
        Liked Citre Shine Moisture Burst Shampoo - 16 fl oz Brand: Citre Shine Price: $23.00
Liked Bath &amp; Body Works Ile De Tahiti Moana Coconut Vanille Moana Body Wash with Tamanoi 8.5 oz Brand: Bath & Body Works Price: 
Liked Bath &amp; Body Works Ile De Tahiti Moana Coconut Vanille Moana Body Wash with Tamanoi 8.5 oz Brand: Bath & Body Works Price: 
Liked Bath &amp; Body Works Ile De Tahiti Moana Coconut Vanille Moana Body Wash with Tamanoi 8.5 oz Brand: Bath & Body Works Price: 
Liked Bath &amp; Body Works Ile De Tahiti Moana Coconut Vanille Moana Body Wash with Tamanoi 8.5 oz Brand: Bath & Body Works Price: 
As an expert beauty product recommender, do the following steps -
    1. An